In [81]:
# iMPORT PACKAGES
import pandas as pd
import datetime as dt
import numpy as np
import math
from datetime import timedelta
from statsmodels.tsa.vector_ar.var_model import VAR
import warnings
warnings.filterwarnings('ignore')


In [82]:
data = pd.read_csv("out_without_nan_mood_target.csv")
data['date'] = pd.to_datetime(data['date'])

In [83]:
df = data.dropna(subset = ['mood_mean','mood_mean_TARGET'] )

In [105]:
# Create test and train set
length_train = 200
index_test = np.random.choice(np.arange(len(df)), length_train, False)

test = df.iloc[index_test]
train = df.drop(df.index[index_test])

In [109]:
def predictMoodPerson(row):    
    #get train data
    train_data_person = df.loc[(df['id'] == row.id) & (df['date'] < row.date)]
    train_data_person_correct = train_data_person[["circumplex.valence_mean", 'mood_mean','mood_mean_TARGET']]
    
    prediction_mood_target = None
    #fit the model
    if len(train_data_person_correct) > 3:
        model = VAR(endog=train_data_person_correct)
        model_fit = model.fit()
    
        # make prediction on validation
        prediction = model_fit.forecast(model_fit.y, steps=1)
        prediction_mood_target = prediction[0][2]
    
    return prediction_mood_target    

In [110]:
# Create empty df to save solutions
solutions = pd.DataFrame(data={'id': [], 'date': [], 'mood_mean_target': [], 'predicted_mood_mean_target':[]})

In [111]:
# Find for every row in test set the predicted mood
for index, row in test.iterrows():
    prediction_mood_target = predictMoodPerson(row)
    
    #add solution to solution df
    if prediction_mood_target is not None:
        solutions = solutions.append({'id': row.id, 'date': row.date, 'mood_mean_target': row.mood_mean_TARGET,
                                      'predicted_mood_mean_target': prediction_mood_target}, ignore_index=True)

In [130]:
#solutions.head(33)
# Gaat fout bij 31 AS14.24 2014-04-19 7.800000 48.954000

In [131]:
def calculateAccMSQ(solutions, name_target,name_predicted):
    correct = 0
    squared_error = 0

    for index, row in solutions.iterrows():
        squared_error += math.pow(row[name_target]-row[name_predicted],2)
        if (row[name_target] == row[name_predicted]):
            correct = correct + 1
        
    accuracy = correct/ len(solutions.index)
    msq = squared_error / len(solutions.index)
    return accuracy, msq
    

In [132]:
accuracy, msq = calculateAccMSQ(solutions, "mood_mean_target", "predicted_mood_mean_target")

print(accuracy, msq)

0.0 9.525162206661223


In [133]:
solutions2 = solutions.round({'mood_mean_target': 0, 'predicted_mood_mean_target': 0})
accuracy, msq = calculateAccMSQ(solutions2, "mood_mean_target", "predicted_mood_mean_target")
print(accuracy, msq)

0.5608465608465608 9.624338624338625
